# Verify the distributions are correct for Gaussian noise

To verify that the median-marginalised distributions are correct, we simulate Gaussian noise with estimated PSDs and show that they follow the expected distributions.

This will reproduce Figures 2-4 of [arXiv:2006.05292](https://arxiv.org/pdf/2006.05292.pdf)

In [ ]:
%pylab inline

from scipy.signal.spectral import _median_bias
from scipy.signal.windows import tukey
from scipy.stats import binom, expon, f, norm, t
from tqdm import tqdm
import warnings

from median_marginalized.distributions import (
    median_marginalised, median_marginalised_likelihood
)

warnings.filterwarnings('ignore')

In [ ]:
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.serif"] = "Computer Modern Roman"
mpl.rcParams["font.size"] = 20
mpl.rcParams["text.usetex"] = True
mpl.rcParams["grid.alpha"] = 0
mpl.rcParams['text.latex.preamble'] = r'\newcommand{\mathdefault}[1][]{}'

# Define functions to simulate data and estimate PSDs

In [ ]:
def forward_backward_fft_with_window(data, tukey_alpha):
    time_domain_data = np.fft.irfft(data, axis=-1)
    window = tukey(time_domain_data.shape[-1], tukey_alpha)
    return np.fft.rfft(time_domain_data * window, axis=-1)

def generate_data_and_psds(n_average, n_samples, true_psd, tukey_alpha=0, do_ffts=True):
    alpha = _median_bias(n_average)

    real_data = np.array([np.random.normal(0, 1, n_samples) * true_psd ** 0.5 for _ in range(n_average + 1)])
    imag_data = np.array([np.random.normal(0, 1, n_samples) * true_psd ** 0.5 for _ in range(n_average + 1)])
    all_data = np.array([real + 1j * imag for real, imag in zip(real_data, imag_data)])
    if not do_ffts and tukey_alpha > 0:
        print("Not doing ffts although non-zero Tukey alpha passed.")
    if do_ffts:
        all_data = forward_backward_fft_with_window(
            data=all_data, tukey_alpha=tukey_alpha
        )
    mean_psd = np.mean(abs(all_data[1:]) ** 2 / 2, axis=0)
    median_psd = np.median(abs(all_data[1:]) ** 2 / 2, axis=0) / alpha
    return all_data, mean_psd, median_psd

# Find the expected distribution of the Anderson-Darling statistic

The expected distribution of the Anderson-Darling statistic does not have a known distribution.

We therefore numerically estimate the distribution.

In [ ]:
def anderson_darling_sample(values, cdf):
    """Compute the Anderson-Darling statistics from a CDf and samples"""
    return (
        - len(values)
        - np.sum(
            (2 * np.arange(1, len(values) + 1) - 1)
            * (np.log(cdf(values)) + np.log(1 - cdf(np.flipud(values))))
        )
        / len(values)
    )

In [ ]:
def generate_expected_anderson_darlings(n_realisations, n_samples, batch_size=32, tukey_alpha=0.1):
    window_factor = 1 - 5 * tukey_alpha / 8
    anderson_darlings = list()
    for ii in tqdm(range(n_realisations // batch_size + 1)):
        real_data = np.array([np.random.normal(0, 1, n_samples) for _ in range(batch_size)])
        imag_data = np.array([np.random.normal(0, 1, n_samples) for _ in range(batch_size)])
        all_data = np.array([real + 1j * imag for real, imag in zip(real_data, imag_data)])
        all_data = forward_backward_fft_with_window(all_data, tukey_alpha) / window_factor ** 0.5
        for data in all_data:
            sorted_data = np.sort(np.concatenate([data.real, data.imag]))
            anderson_darlings.append(
                anderson_darling_sample(sorted_data, norm(loc=0, scale=1).cdf)
            )
    return anderson_darlings[:n_realisations]


def generate_expected_anderson_darlings_power(n_realisations, n_samples, batch_size=32, tukey_alpha=0.1):
    window_factor = 1 - 5 * tukey_alpha / 8
    anderson_darlings = list()
    for ii in tqdm(range(n_realisations // batch_size + 1)):
        real_data = np.array([np.random.normal(0, 1, n_samples) for _ in range(batch_size)])
        imag_data = np.array([np.random.normal(0, 1, n_samples) for _ in range(batch_size)])
        all_data = np.array([real + 1j * imag for real, imag in zip(real_data, imag_data)])
        all_data = forward_backward_fft_with_window(all_data, tukey_alpha) / window_factor ** 0.5
        for data in all_data:
            sorted_data = np.sort(np.abs(data.real + 1j * data.imag))
            anderson_darlings.append(
                anderson_darling_sample(sorted_data ** 2, expon(scale=2).cdf)
            )
    return anderson_darlings[:n_realisations]

In [ ]:
expected_anderson_darlings = generate_expected_anderson_darlings(n_realisations=16384 // 16, n_samples=16384)
expected_anderson_darlings_power = generate_expected_anderson_darlings_power(n_realisations=16384 // 16, n_samples=16384)
plt.figure(figsize=(8, 5))
plt.plot(np.sort(expected_anderson_darlings), np.linspace(1, 0, len(expected_anderson_darlings)))
plt.plot(np.sort(expected_anderson_darlings_power), np.linspace(1, 0, len(expected_anderson_darlings_power)))
plt.yscale("log")
plt.xlabel("$A^{2}$")
plt.xlabel("$S(A^{2})$")
plt.tight_layout()
plt.show()
plt.close()

# Simulate the data

We loop over a number of averages and segment durations, generate data and test the distribution matches the expected.

First, we define a bunch of plotting functions.

In [ ]:
def plot_pp_confidence_levels(confidence_interval, confidence_interval_alpha, ax=None, n_samples=None):
    if ax is None:
        ax = plt.gca()
    for ci, alpha in zip(confidence_interval, confidence_interval_alpha):
        if n_samples is None:
            n_samples = len(all_data[0])
        _x_values = np.linspace(0, 1, 1000)
        edge_of_bound = (1. - ci) / 2.
        lower = binom.ppf(1 - edge_of_bound, n_samples, _x_values) / n_samples - _x_values
        upper = binom.ppf(edge_of_bound, n_samples, _x_values) / n_samples - _x_values
        lower[0] = 0
        upper[0] = 0
        ax.fill_between(_x_values, lower, upper, alpha=alpha, color='k')


def plot_anderson_darlings(
    exact_ads, mean_ads, median_ads, mean_bad_ads, median_bad_ads, expected_ads
):
    plt.figure(figsize=(8, 5))
    plt.plot(
        np.sort(exact_ads),
        np.linspace(1, 0, len(exact_ads)),
        color="C0",
        label="Exact"
    )
    plt.plot(
        np.sort(mean_ads),
        np.linspace(1, 0, len(mean_ads)),
        color="C1",
        label="Mean marginalised"
    )
    plt.plot(
        np.sort(median_ads),
        np.linspace(1, 0, len(median_ads)),
        color="C2",
        label="Median marginalised"
    )
    plt.plot(
        np.sort(mean_bad_ads),
        np.linspace(1, 0, len(mean_bad_ads)),
        color="C3",
        label="Mean unmarginalised"
    )
    plt.plot(
        np.sort(median_bad_ads),
        np.linspace(1, 0, len(median_bad_ads)),
        color="C4",
        label="Median unmarginalised"
    )
    ads = np.sort([
        expected_ads[ii * len(mean_ads): (ii + 1) * len(mean_ads)]
        for ii in range(len(expected_ads) // len(mean_ads))
    ], axis=-1)
    for interval in [0.68, 0.95]:
        plt.fill_betweenx(
            np.linspace(1, 0, len(ads[0])),
            np.quantile(ads, 0.5 - interval / 2, axis=0),
            np.quantile(ads, 0.5 + interval / 2, axis=0),
            color="k",
            alpha=0.1
        )
    ax = plt.gca()
    ax.set_xlim(0, 5)
    ax.set_xlabel("$A^2$")
    ax.set_ylabel("$S(A^2)$")
    ax.set_yscale("log")
    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.show()
    plt.close()


def plot_whitened_data(exact_whitened_data, mean_whitened_data, median_whitened_data):
    plt.figure(figsize=(8, 5))
    plt.plot(np.linspace(-10, 10, 1000), norm(loc=0, scale=1).pdf(np.linspace(-10, 10, 1000)), color="C0", label="Known")
    plt.plot(np.linspace(-10, 10, 1000), t(loc=0, scale=1, df=2 * n_average).pdf(np.linspace(-10, 10, 1000)), color="C1", label="Mean")
    plt.plot(np.linspace(-10, 10, 1000), median_marginalised(loc=0, scale=1, n_average=n_average).pdf(np.linspace(-10, 10, 1000)), color="C2", label="Median")
    
    plt.hist(np.hstack([exact_whitened_data.real, exact_whitened_data.imag]), bins=200, histtype="step", alpha=1, density=True, color="C0")
    plt.hist(np.hstack([mean_whitened_data.real, mean_whitened_data.imag]), bins=200, histtype="step", alpha=1, density=True, color="C1")
    plt.hist(np.hstack([median_whitened_data.real, median_whitened_data.imag]), bins=200, histtype="step", alpha=1, density=True, color="C2")
    ax = plt.gca()
    ax.set_xlabel("$\\tilde{\\nu}$")
    ax.set_ylabel("$p(\\tilde{\\nu})$")
    ax.set_yscale("log")
    if n_average >= 31:
        ax.set_xlim(-6, 6)
    elif n_average >= 15:
        ax.set_xlim(-8, 8)
    else:
        ax.set_xlim(-10, 10)
    ax.set_ylim(1e-5, 1)
    ax.legend(loc="lower center")
    plt.tight_layout()
    plt.show()
    plt.close()


def plot_whitened_power(exact_whitened_data, mean_whitened_data, median_whitened_data):
    plt.figure(figsize=(8, 5))
    plt.plot(np.linspace(0, 10, 1000) ** 2, expon(scale=2).pdf(np.linspace(0, 10, 1000) ** 2), color="C0", label="Known")
    plt.plot(np.linspace(0, 10, 1000) ** 2, f(loc=0, scale=2, dfn=2, dfd=2 * n_average).pdf(np.linspace(0, 10, 1000) ** 2), color="C1", label="Mean")
    plt.plot(np.linspace(0, 10, 1000) ** 2, median_marginalised_likelihood(loc=0, scale=1, n_average=n_average).pdf(np.linspace(0, 10, 1000) ** 2), color="C2", label="Median")
    
    plt.hist(abs(exact_whitened_data) ** 2, bins=200, histtype="step", alpha=1, density=True, color="C0")
    plt.hist(abs(mean_whitened_data) ** 2, bins=200, histtype="step", alpha=1, density=True, color="C1")
    plt.hist(abs(median_whitened_data) ** 2, bins=200, histtype="step", alpha=1, density=True, color="C2")
    ax = plt.gca()
    ax.set_xlabel("$|\\tilde{\\nu}|^2$")
    ax.set_ylabel("$\\mathcal{L}(\\tilde{\\nu})$")
    ax.set_yscale("log")
    if n_average >= 31:
        ax.set_xlim(0, 30)
    elif n_average >= 15:
        ax.set_xlim(0, 40)
    else:
        ax.set_xlim(0, 70)
    ax.set_ylim(1e-5, 1)
    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.show()
    plt.close()


def plot_ppp_whitened(mean_data, median_data):
    plt.figure(figsize=(8, 5))
    ax = plt.gca()
    _data = np.hstack([mean_data.real, mean_data.imag])
    plot_pp_confidence_levels(confidence_interval, confidence_interval_alpha, ax, n_samples=len(_data) * 2)
    plt.plot(
        np.linspace(0, 1, len(_data)),
        t(loc=0, scale=1, df=2 * n_average).cdf(np.sort(_data))
        - np.linspace(0, 1, len(_data)),
        color="C1",
        alpha=1,
        label="Mean"
    )
    _data = np.hstack([median_data.real, median_data.imag])
    plt.plot(
        np.linspace(0, 1, len(_data)),
        median_marginalised(loc=0, scale=1, n_average=n_average).cdf(np.sort(_data))
        - np.linspace(0, 1, len(_data)),
        color="C2",
        alpha=1,
        label="Median"
    )
    _data = np.hstack([mean_data.real, mean_data.imag])
    plt.plot(
        np.linspace(0, 1, len(_data)),
        norm(loc=0, scale=1).cdf(np.sort(_data))
        - np.linspace(0, 1, len(_data)),
        color="C3",
        alpha=1,
        label="Mean unmarginalised"
    )
    _data = np.hstack([median_data.real, median_data.imag])
    plt.plot(
        np.linspace(0, 1, len(_data)),
        norm(loc=0, scale=1).cdf(np.sort(_data))
        - np.linspace(0, 1, len(_data)),
        color="C4",
        alpha=1,
        label="Median unmarginalised"
    )
        
    ax.set_xlabel("$\\Phi(\\tilde{\\nu})$")
    ax.set_xlim(0, 1)
    ax.set_yscale("linear")
    ax.ticklabel_format(axis="y", style="plain")
    ax.set_ylabel("$\\Phi_{s}(\\tilde{\\nu}) - \\Phi(\\tilde{\\nu})$")
    plt.legend(loc="upper left", ncol=2)
    plt.tight_layout()
    plt.show()
    plt.close()


def plot_ppp_power(mean_data, median_data):
    plt.figure(figsize=(8, 5))
    ax = plt.gca()
    _data = abs(mean_data)
    plot_pp_confidence_levels(confidence_interval, confidence_interval_alpha, ax, n_samples=len(_data))
    plt.plot(
        np.linspace(0, 1, len(_data)),
        f(loc=0, scale=2, dfn=2, dfd=2 * n_average).cdf(np.sort(_data) ** 2)
        - np.linspace(0, 1, len(_data)),
        color="C1",
        label="Mean"
    )
    _data = abs(median_data)
    plt.plot(
        np.linspace(0, 1, len(_data)),
        median_marginalised_likelihood(loc=0, scale=1, n_average=n_average).cdf(np.sort(_data) ** 2)
        - np.linspace(0, 1, len(_data)),
        color="C2",
        label="Median"
    )
    _data = abs(mean_data)
    plt.plot(
        np.linspace(0, 1, len(_data)),
        expon(scale=2).cdf(np.sort(_data) ** 2)
        - np.linspace(0, 1, len(_data)),
        color="C3",
        alpha=1,
        label="Mean unmarginalised"
    )
    _data = abs(median_data)
    plt.plot(
        np.linspace(0, 1, len(_data)),
        expon(scale=2).cdf(np.sort(_data) ** 2)
        - np.linspace(0, 1, len(_data)),
        color="C4",
        alpha=1,
        label="Median unmarginalised"
    )
        
    ax.set_xlabel("$\\Phi(|\\tilde{\\nu}|)$")
    ax.set_xlim(0, 1)
    ax.set_yscale("linear")
    ax.ticklabel_format(axis="y", style="plain")
    ax.set_ylabel("$\\Phi_{s}(|\\tilde{\\nu}|) - \\Phi(|\\tilde{\\nu}|)$")
    plt.legend(loc="upper left", ncol=2)
    plt.tight_layout()
    plt.show()
    plt.close()


In [ ]:
sampling_frequency = 4096
tukey_alpha = 0.1
window_factor = 1 - 5 * tukey_alpha / 8

legend_position = "lower right"
confidence_interval = [0.68, 0.95, 0.997]
confidence_interval_alpha = [0.1, 0.1, 0.1]
hist_kwargs = dict(
    histtype="step", bins=100, density=True
)

total_samples = 1024

for n_average, duration in zip([31, 63, 127], [8, 4, 4]):
    n_samples = int(sampling_frequency * duration / 2 + 1)
    true_psd = 10 ** np.linspace(-40, -45, n_samples)
    true_psd[:duration * 10] = 0
    true_psd[-1] = 0
    mask = np.ones_like(true_psd, dtype="bool")
    mask[:duration * 20] = False
    mask[-1] = False

    exact_ads = [0]
    mean_ads = [0]
    median_ads = [0]
    mean_bad_ads = [0]
    median_bad_ads = [0]
    exact_ads_power = [0]
    mean_ads_power = [0]
    median_ads_power = [0]
    mean_bad_ads_power = [0]
    median_bad_ads_power = [0]
    all_exact_whitened_data = list()
    all_mean_whitened_data = list()
    all_median_whitened_data = list()

    for _ in tqdm(range(total_samples // duration)):
        all_data, mean_psd, median_psd = generate_data_and_psds(
            n_average, n_samples, true_psd,
            tukey_alpha=tukey_alpha, do_ffts=True
        )
        all_data = all_data[:, mask]
        alpha = _median_bias(n_average)
        analysis_data = all_data[-1]
        mean_psd = np.mean(abs(all_data[:-1]) ** 2 / 2, axis=0)
        median_psd = np.median(abs(all_data[:-1]) ** 2 / 2, axis=0) / alpha
        
        exact_whitened_data = np.sort(
            analysis_data.real / true_psd[mask] ** 0.5 / window_factor ** 0.5
        )
        mean_whitened_data = np.sort(analysis_data.real / mean_psd ** 0.5)
        median_whitened_data = np.sort(analysis_data.real / median_psd ** 0.5)
        median_whitened_data = median_whitened_data[abs(median_whitened_data) < 20]

        all_exact_whitened_data.append(analysis_data / true_psd[mask] ** 0.5 / window_factor ** 0.5)
        all_mean_whitened_data.append(analysis_data / mean_psd ** 0.5)
        all_median_whitened_data.append(analysis_data / median_psd ** 0.5)

        exact_ads.append(anderson_darling_sample(
            exact_whitened_data,
            norm(loc=0, scale=1).cdf
        ))
        mean_ads.append(anderson_darling_sample(
            mean_whitened_data,
            t(loc=0, scale=1, df=2 * n_average).cdf
        ))
        median_ads.append(anderson_darling_sample(
            median_whitened_data,
            median_marginalised(loc=0, scale=1, n_average=n_average).cdf
        ))
        mean_bad_ads.append(anderson_darling_sample(
            mean_whitened_data,
            norm(loc=0, scale=1).cdf
        ))
        median_bad_ads.append(anderson_darling_sample(
            median_whitened_data,
            norm(loc=0, scale=1).cdf
        ))

        exact_ads_power.append(anderson_darling_sample(
            np.sort(abs(analysis_data / true_psd[mask] ** 0.5 / window_factor ** 0.5)) ** 2,
            expon(scale=2).cdf
        ))
        mean_ads_power.append(anderson_darling_sample(
            np.sort(abs(analysis_data / mean_psd ** 0.5)) ** 2,
            f(loc=0, scale=2, dfn=2, dfd=2 * n_average).cdf
        ))
        median_ads_power.append(anderson_darling_sample(
            np.sort(abs(analysis_data / median_psd ** 0.5)) ** 2,
            median_marginalised_likelihood(loc=0, scale=1, n_average=n_average).cdf
        ))
        mean_bad_ads_power.append(anderson_darling_sample(
            np.sort(abs(analysis_data / mean_psd ** 0.5)) ** 2,
            expon(scale=2).cdf
        ))
        median_bad_ads_power.append(anderson_darling_sample(
            np.sort(abs(analysis_data / median_psd ** 0.5)) ** 2,
            expon(scale=2).cdf
        ))
        
    all_exact_whitened_data = np.concatenate(all_exact_whitened_data)
    all_mean_whitened_data = np.concatenate(all_mean_whitened_data)
    all_median_whitened_data = np.concatenate(all_median_whitened_data)

    plot_whitened_data(
        all_exact_whitened_data, all_mean_whitened_data, all_median_whitened_data
    )
    plot_whitened_power(
        all_exact_whitened_data, all_mean_whitened_data, all_median_whitened_data
    )
    
    plot_ppp_whitened(all_mean_whitened_data, all_median_whitened_data)
    plot_ppp_power(all_mean_whitened_data, all_median_whitened_data)

    plot_anderson_darlings(
        exact_ads,
        mean_ads,
        median_ads,
        mean_bad_ads,
        median_bad_ads,
        expected_anderson_darlings
    )
    plot_anderson_darlings(
        exact_ads_power,
        mean_ads_power,
        median_ads_power,
        mean_bad_ads_power,
        median_bad_ads_power,
        expected_anderson_darlings_power
    )